# Overview

A Kafka cluster with 1 zk and 2 kafka brokers


The solution is based on the docker-compose files at [6za/kafka](https://github.com/6za/kafka).

The files are enhanced by setup to add network to allow this solution to work over multiple docker hosts without meshing them into a single virtual network. 

This setup is using raspberry servers

In [1]:
import pandas as pd
import yaml
import os
import subprocess

## Nodes
- Zookeeper Node
- Kafka Node

In [2]:
hosts = pd.read_csv("../common/hosts.csv")
supressed_columns = ['ip','user']
raspi_zk_hosts = hosts[(hosts.hostname == "pi-node251")]
raspi_kafka_hosts = hosts[(hosts.hostname == "pi-node250") | (hosts.hostname == "pi-node249") ]

In [3]:
raspi_zk_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
15,pi-node251,armv7l,0


In [4]:
raspi_kafka_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
13,pi-node249,armv7l,0
14,pi-node250,armv7l,0


## Clone Repo

In [5]:
%%bash
mkdir ~/repos
cd ~/repos
rm -rf ~/repos/kafka
git clone https://github.com/6za/kafka.git

mkdir: cannot create directory '/root/repos': File exists
Cloning into 'kafka'...


## Create host lists

### Zookeeper Host List

- Zookeeper and all kafka nodes need to be visible to allow brokers to work

In [6]:

zk_ip = raspi_zk_hosts.iloc[0]['ip']
zk_config = 'zookeeper:%(ip)s' %  {"ip": zk_ip}
kafka_hosts_list = [zk_config]

for index, row in raspi_kafka_hosts.iterrows():
        hostname = 'kafka-' + row['hostname']
        config = '%(hostname)s:%(ip)s' %  {"hostname": hostname, "ip": row['ip']}
        kafka_hosts_list.append(config)

len(kafka_hosts_list)

3

## Deploy Zookeeper

- Deploy as is no modification on files

In [7]:
for index, row in raspi_zk_hosts.iterrows():
    print('\x1b[1;35m'+ row['hostname']+'\x1b[0m')
    docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
    command = "docker-compose -f ~/repos/kafka/docker-compose-zk-1node.yaml down"  
    result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')        
    command = "docker-compose -f ~/repos/kafka/docker-compose-zk-1node.yaml up -d --build"  
    result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')        
    print("Zookeeper Started")

pi-node251
Zookeeper Started


## Deploy Kafka Nodes

In [9]:
!sleep 30
node_count = 0
with open('/root/repos/kafka/docker-compose-kafka-1node.yaml',"r") as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)
    config['services']['kafka']['extra_hosts'] = kafka_hosts_list
    for index, row in raspi_kafka_hosts.iterrows():
        node_count = node_count + 1
        config['services']['kafka']['hostname'] = 'kafka-' + row['hostname']
        with open(r'docker-compose-kafka-current.yaml', 'w') as outputfile:
            documents = yaml.dump(config, outputfile)    
        print('\x1b[1;35m'+ row['hostname']+'\x1b[0m')
        docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
        command = "docker-compose -f docker-compose-kafka-current.yaml down" 
        result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')        
        command = "docker-compose  -f docker-compose-kafka-current.yaml up -d --build" 
        result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')        
        print("  Kafka started:" + row['hostname'])
        !sleep 5
        command = "docker ps"  
        result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')        
        print(result)

        


pi-node249
  Kafka started:pi-node249
CONTAINER ID        IMAGE                        COMMAND                  CREATED             STATUS              PORTS                    NAMES
eddc5d626ce2        12-kafka-cluster_kafka       "bash -c '/opt/kafka…"   9 seconds ago       Up 5 seconds        0.0.0.0:9092->9092/tcp   12-kafka-cluster_kafka_1
d367b00450b5        prom/node-exporter:v0.18.0   "/bin/node_exporter …"   5 days ago          Up 5 days           0.0.0.0:9100->9100/tcp   nodeexporter

pi-node250
  Kafka started:pi-node250
CONTAINER ID        IMAGE                        COMMAND                  CREATED             STATUS              PORTS                    NAMES
8393366e6031        12-kafka-cluster_kafka       "bash -c '/opt/kafka…"   8 seconds ago       Up 5 seconds        0.0.0.0:9092->9092/tcp   12-kafka-cluster_kafka_1
8ea920efccdf        prom/node-exporter:v0.18.0   "/bin/node_exporter …"   5 days ago          Up 5 days           0.0.0.0:9100->9100/tcp   nodeexporter

